To Do:
-Cycle through directory to process all zip files
-Process all files in zip file

-Load table of what items are associated with what items, trim table to only items created after 2015
-Determine where data stands
-Calculate cost per packet, divide cost per packet by number of items
-Scope out messy/unmatched data

In [135]:
import pandas as pd
import numpy
import xlrd
from zipfile import ZipFile

In [136]:
import numpy
import xlrd
from zipfile import ZipFile

def open_zip(datafile):
    with ZipFile('{0}'.format(datafile), 'r') as myzip:
        myzip.extractall()        


In [144]:
def process_timesheet(unzipped_datafile, timesheet):
    workbook = xlrd.open_workbook(unzipped_datafile)
    sheet = workbook.sheet_by_index(0)

    #Process first tab of timesheet
    #On the first tab, save cell B3 as the contractor name
    name = sheet.cell_value(2, 1)
    #On the first tab, save cell D18 as the contractor rate
    payrate = sheet.cell_value(17,3)
    if payrate == "Hourly \nRate":
        payrate = sheet.cell_value(18,3)  #likely a more elegant solution, but brute forcing for now
    date = str(int(sheet.cell_value(2,4))) + "-" + str(int(sheet.cell_value(2,5))) + "-" + str(int(sheet.cell_value(2,6)))

    #Process second tab of Excel spreadsheet
    sheet = workbook.sheet_by_index(1)
    #On the second tab, starting with cell A6:E6, save each row until you encounter an empty row
    row = 5
    hours = sheet.cell_value(row,0)
    activity = sheet.cell_value(row, 1)
    description = sheet.cell_value(row, 4)
    while hours != '' and activity != "Total":
        cost = payrate * hours
        timesheet = timesheet.append({'Name':name, 'Payrate':payrate, 'Invoice Date':date, 'Hours':hours, 'Cost': cost, 'Activity':activity, 'Description':description}, ignore_index=True)
        row += 1
        hours = sheet.cell_value(row,0)
        activity = sheet.cell_value(row, 1)
        description = sheet.cell_value(row, 4)
    return timesheet

In [145]:
import glob

timesheet = pd.DataFrame([],columns=['Name', 'Invoice Date','Payrate','Hours', 'Cost', 'Activity','Description'])

for file in glob.glob("*.zip"):    # only loops over zip files
        open_zip(file)
        
#apply process_timesheet to all unzipped Excel files
for file in glob.glob("*.xlsx"):
        timesheet = process_timesheet(file, timesheet)

In [146]:
print timesheet

               Name Invoice Date  Payrate  Hours   Cost  \
0      Greg Klinger      7-16-16     40.0   1.00   40.0   
1      Greg Klinger      7-16-16     40.0   1.00   40.0   
2      Greg Klinger      7-16-16     40.0   1.00   40.0   
3      Greg Klinger      7-16-16     40.0   0.50   20.0   
4      Greg Klinger       8-1-16     40.0   2.00   80.0   
5      Greg Klinger       8-1-16     40.0   2.00   80.0   
6      Greg Klinger      8-16-16     40.0   6.00  240.0   
7      Greg Klinger      8-16-16     40.0   2.00   80.0   
8      Greg Klinger      8-16-16     40.0   1.00   40.0   
9      Greg Klinger      8-16-16     40.0   0.25   10.0   
10     Greg Klinger      8-16-16     40.0   2.00   80.0   
11     Greg Klinger      8-16-16     40.0   2.00   80.0   
12     Greg Klinger       9-1-16     40.0   2.00   80.0   
13     Greg Klinger       9-1-16     40.0   3.00  120.0   
14     Greg Klinger       9-1-16     40.0   4.00  160.0   
15     Greg Klinger       9-1-16     40.0   2.25   90.0 

In [147]:
timesheet.groupby('Activity').sum()

,Payrate,Hours,Cost
Activity,,,
,135.0,3.000,135.00
Alignment - New,14736.0,1815.250,69485.75
Alignment - Review,1141.0,295.250,11105.75
Documentation,2691.0,1182.000,45092.50
Editorial Data Entry,1050.0,846.000,27463.75
Editorial Review,8512.0,580.250,18704.50
File Building,595.0,117.500,4112.50
Item Orders,409.0,40.500,2055.25
Item Review - Existing,2620.0,183.750,7478.00


In [148]:
timesheet.groupby('Description').sum()

,Payrate,Hours,Cost
Description,,,
,225.0,32.00,1337.50
Art for Math-CCSS-Spr2016-TEI-Gr8-10,25.0,0.50,12.50
Art for Pre-Delivery Image QC Board,25.0,8.00,200.00
Art for SCI-NC-40 Earth Science,25.0,1.00,25.00
Art for SCI-NC-42,25.0,0.25,6.25
Art for Sci-NGSS-29-Physical Science,25.0,0.25,6.25
"ELA, Science & Math reviews",32.0,95.00,3040.00
"ELA, Science & Math reviews, new translations",32.0,93.00,2976.00
"ELA, Science,& Math reviews",32.0,91.00,2912.00


Processing tasks:
    -replace words that begin with lowercase letter with uppercase letter in Activity
    -Load table of what items are associated with what items, trim table to only items created after 2015
    -determine whether line item is art, item, passage, or packet
    -classify description is useful for calculations
    -Calculate item codes
    -Calculate passage codes
    -Calculate packets
